In [ ]:
import numpy as np
import pymc3 as pm
import theano.tensor as tt

%matplotlib inline
from IPython.core.pylabtools import figsize
from matplotlib import pyplot as plt
figsize(11, 9)

import scipy.stats as stats

### Recreating count data example

In [ ]:
count_data = np.loadtxt('./Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/Chapter1_Introduction/data/txtdata.csv') 

In [ ]:
avg_texts = count_data.mean()

In [ ]:
with pm.Model() as model:
    idx = np.arange(len(count_data))
    tau = pm.DiscreteUniform("tau", lower=0, upper=len(count_data) - 1)
    lambda1 = pm.Exponential("lambda1", 1./avg_texts)
    lambda2 = pm.Exponential("lambda2", 1./avg_texts)
    lambda_ = pm.math.switch(tau >= idx, lambda1, lambda2)
    count = pm.Poisson("count", lambda_, observed=count_data)

In [ ]:
with model:
    step = pm.Metropolis()
    trace = pm.sample(20000, step=step)

In [ ]:
burnt_trace = trace[10000:]

In [ ]:
ax = plt.subplot(311)
plt.hist(burnt_trace['lambda1'], histtype='stepfilled', bins=35, normed=True)
plt.xlim([15,30])
plt.title(r"Posterior of $\lambda_1,\lambda_2$ and $\tau$")

ax = plt.subplot(312)
plt.hist(burnt_trace['lambda2'], histtype='stepfilled', bins=35, normed=True)
plt.xlim([15,30])

ax = plt.subplot(313)
plt.hist(burnt_trace['tau'], histtype='stepfilled', bins=35, normed=True)
plt.xlabel('Switching day')
plt.ylabel('Probability')

### Recreating Bayesian A/B testing 

In [ ]:
true_conversion_rate_A = 0.05
true_conversion_rate_B = 0.03
N_A = 1500
N_B = 750

observations_A = stats.bernoulli.rvs(true_conversion_rate_A, size=N_A)
observations_B = stats.bernoulli.rvs(true_conversion_rate_B, size=N_B)

In [ ]:
with pm.Model() as ab_testing:
    conversion_rate_A = pm.Uniform("conversion_rate_A", lower=0, upper=1)
    conversion_rate_B = pm.Uniform("conversion_rate_B", lower=0, upper=1)
    
    delta = pm.Deterministic("delta", conversion_rate_A - conversion_rate_B)
    
    conversions_A = pm.Bernoulli("conversions_A", conversion_rate_A, observed=observations_A)
    conversions_B = pm.Bernoulli("conversions_B", conversion_rate_B, observed=observations_B)

In [ ]:
with ab_testing:
    step = pm.Metropolis()
    trace = pm.sample(20000, step=step)
    burnt_trace = trace[10000:]

In [ ]:

ax = plt.subplot(311)
plt.title('Posterior distribution for conversion rate A')
plt.hist(burnt_trace['conversion_rate_A'], histtype='stepfilled', normed=True)
plt.axvline(0.05, color='black')

ax = plt.subplot(312)
plt.title('Posterior distribution for conversion rate A')
plt.hist(burnt_trace['conversion_rate_B'], histtype='stepfilled', normed=True)
plt.axvline(0.03, color='black')
plt.tight_layout()
ax = plt.subplot(313)
plt.title('Posterior distribution for conversion rate $\Delta$')
plt.hist(burnt_trace['delta'], histtype='stepfilled', normed=True)
plt.axvline(0.02, color='black')


In [ ]:
p = sum(burnt_trace['conversion_rate_A'] > burnt_trace['conversion_rate_B']) / len(burnt_trace['conversion_rate_A'])

In [ ]:
print("Probability of A being superior to B is {:.2%}".format(p))